## 📦 Step 1: Install Dependencies
Run this cell first to install all required packages.

In [1]:
# For VS Code local execution, use pip install (without !)
# Run this in terminal if packages are not installed:
# pip install librosa numpy soundfile pesq pystoi

import subprocess
import sys

packages = ['librosa', 'numpy', 'soundfile', 'pesq', 'pystoi']

for package in packages:
    try:
        __import__(package)
        print(f"✅ {package} already installed")
    except ImportError:
        print(f"📦 Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"✅ {package} installed")

print("\n✅ All dependencies ready!")

✅ librosa already installed
✅ numpy already installed
✅ soundfile already installed
✅ pesq already installed
✅ numpy already installed
✅ soundfile already installed
✅ pesq already installed
✅ pystoi already installed

✅ All dependencies ready!
✅ pystoi already installed

✅ All dependencies ready!


## 📚 Step 2: Import Libraries

In [2]:
import os
import json
import numpy as np
import librosa
import soundfile as sf
from pathlib import Path
from typing import Dict, Optional, Tuple, Any
import warnings

warnings.filterwarnings('ignore')

# Check available packages
try:
    from pesq import pesq
    PESQ_AVAILABLE = True
    print("✅ PESQ is available")
except ImportError:
    PESQ_AVAILABLE = False
    print("⚠️ PESQ not installed")

try:
    from pystoi import stoi
    STOI_AVAILABLE = True
    print("✅ STOI is available")
except ImportError:
    STOI_AVAILABLE = False
    print("⚠️ PYSTOI not installed")

# Optional DNSMOS
try:
    import dnsmos
    DNSMOS_AVAILABLE = True
    print("✅ DNSMOS is available")
except ImportError:
    DNSMOS_AVAILABLE = False
    print("ℹ️ DNSMOS not installed (will use PESQ-to-MOS fallback)")

print("\n✅ All imports completed!")

✅ PESQ is available
✅ STOI is available
ℹ️ DNSMOS not installed (will use PESQ-to-MOS fallback)

✅ All imports completed!


## 📁 Step 3: Configure Paths

**Update these paths based on your environment:**
- For **Google Colab**: Use `/content/...` paths
- For **Local**: Use your actual folder paths

In [3]:
# =============================================================================
# 📁 CONFIGURE YOUR PATHS HERE
# =============================================================================
import os
import sys

# Auto-detect environment
print("🔍 Environment Detection:")
print(f"   Platform: {sys.platform}")
print(f"   CWD: {os.getcwd()}")

# =============================================================================
# PATH CONFIGURATION
# =============================================================================

# Base folder
BASE_FOLDER = r"c:\Users\rayba\Downloads\samsung"

# ORIGINAL DATA (clean reference)
ORIGINAL_DATA_FOLDER = os.path.join(BASE_FOLDER, "original_data")
ORIGINAL_AUDIO_FOLDER = os.path.join(ORIGINAL_DATA_FOLDER, "audio_file")
ORIGINAL_METADATA_FOLDER = os.path.join(ORIGINAL_DATA_FOLDER, "audio_metadata")

# RECORDED DATA (to be analyzed and compared with original)
RECORDED_DATA_FOLDER = os.path.join(BASE_FOLDER, "recorded_data")
RECORDED_AUDIO_FOLDER = os.path.join(RECORDED_DATA_FOLDER, "recorded_audio")
RECORDED_METADATA_FOLDER = os.path.join(RECORDED_DATA_FOLDER, "recorded_audio_metadata")

# Output folder
OUTPUT_FOLDER = os.path.join(BASE_FOLDER, "json_output")

# Legacy compatibility
AUDIO_FOLDER = ORIGINAL_AUDIO_FOLDER
METADATA_FOLDER = ORIGINAL_METADATA_FOLDER
CLEAN_FOLDER = None
NOISE_FOLDER = None

# =============================================================================
# SETTINGS
# =============================================================================
TARGET_SR = 16000
SUPPORTED_FORMATS = ('.wav', '.flac', '.mp3', '.ogg', '.m4a')
SPIKE_THRESHOLD_RATIO = 3.0
MIN_SPIKE_DURATION_MS = 1

# =============================================================================
# VERIFY PATHS
# =============================================================================
print("\n📁 Path Verification:")
print("=" * 60)

print("\n📂 ORIGINAL DATA:")
if os.path.exists(ORIGINAL_AUDIO_FOLDER):
    orig_files = [f for f in os.listdir(ORIGINAL_AUDIO_FOLDER) if f.lower().endswith(SUPPORTED_FORMATS)]
    print(f"  ✅ Audio: {ORIGINAL_AUDIO_FOLDER} ({len(orig_files)} files)")
else:
    print(f"  ❌ Audio NOT FOUND: {ORIGINAL_AUDIO_FOLDER}")

if os.path.exists(ORIGINAL_METADATA_FOLDER):
    orig_meta = [f for f in os.listdir(ORIGINAL_METADATA_FOLDER) if f.endswith('.json')]
    print(f"  ✅ Metadata: {ORIGINAL_METADATA_FOLDER} ({len(orig_meta)} files)")
else:
    print(f"  ❌ Metadata NOT FOUND: {ORIGINAL_METADATA_FOLDER}")

print("\n📂 RECORDED DATA:")
if os.path.exists(RECORDED_AUDIO_FOLDER):
    rec_files = [f for f in os.listdir(RECORDED_AUDIO_FOLDER) if f.lower().endswith(SUPPORTED_FORMATS)]
    print(f"  ✅ Audio: {RECORDED_AUDIO_FOLDER} ({len(rec_files)} files)")
else:
    print(f"  ❌ Audio NOT FOUND: {RECORDED_AUDIO_FOLDER}")

if os.path.exists(RECORDED_METADATA_FOLDER):
    rec_meta = [f for f in os.listdir(RECORDED_METADATA_FOLDER) if f.endswith('.json')]
    print(f"  ✅ Metadata: {RECORDED_METADATA_FOLDER} ({len(rec_meta)} files)")
else:
    print(f"  ❌ Metadata NOT FOUND: {RECORDED_METADATA_FOLDER}")

print(f"\n📂 Output: {OUTPUT_FOLDER}")

🔍 Environment Detection:
   Platform: win32
   CWD: c:\Users\rayba\Downloads\samsung

📁 Path Verification:

📂 ORIGINAL DATA:
  ✅ Audio: c:\Users\rayba\Downloads\samsung\original_data\audio_file (20 files)
  ✅ Metadata: c:\Users\rayba\Downloads\samsung\original_data\audio_metadata (20 files)

📂 RECORDED DATA:
  ✅ Audio: c:\Users\rayba\Downloads\samsung\recorded_data\recorded_audio (20 files)
  ✅ Metadata: c:\Users\rayba\Downloads\samsung\recorded_data\recorded_audio_metadata (20 files)

📂 Output: c:\Users\rayba\Downloads\samsung\json_output


## 📤 Step 3b: Upload Audio Files (Google Colab Only)

Run this cell if you're using Google Colab to upload your audio files.

In [4]:
# =============================================================================
# 📤 UPLOAD FILES TO GOOGLE COLAB
# =============================================================================
# Run this cell to upload your audio files

import os

# Create the audio_files folder
os.makedirs("/content/audio_files", exist_ok=True)
print("✅ Created folder: /content/audio_files")

# Option 1: Upload files directly
print("\n📤 Uploading files...")
print("   A file picker dialog will open. Select your .flac files.")
print("   (You can select multiple files at once)")

try:
    from google.colab import files
    uploaded = files.upload()
    
    # Move uploaded files to audio_files folder
    for filename in uploaded.keys():
        os.rename(filename, f"/content/audio_files/{filename}")
        print(f"   ✅ Moved: {filename}")
    
    print(f"\n✅ Uploaded {len(uploaded)} files to /content/audio_files/")
except ImportError:
    print("⚠️ Not running in Google Colab. Skip this cell for local execution.")
except Exception as e:
    print(f"❌ Error: {e}")

# List files in audio_files folder
if os.path.exists("/content/audio_files"):
    files_list = os.listdir("/content/audio_files")
    print(f"\n📂 Files in /content/audio_files: {len(files_list)}")
    for f in files_list[:10]:
        print(f"   - {f}")

✅ Created folder: /content/audio_files

📤 Uploading files...
   A file picker dialog will open. Select your .flac files.
   (You can select multiple files at once)
⚠️ Not running in Google Colab. Skip this cell for local execution.

📂 Files in /content/audio_files: 0


## 🔧 Step 4: Utility Functions

In [13]:
import subprocess
import struct

def load_audio_with_ffmpeg(filepath: str, target_sr: int = TARGET_SR) -> Tuple[np.ndarray, int]:
    """
    Load audio file using FFmpeg from imageio-ffmpeg.
    This works for all formats including MP3.
    """
    import imageio_ffmpeg
    
    ffmpeg_exe = imageio_ffmpeg.get_ffmpeg_exe()
    
    # Use FFmpeg to convert to raw PCM
    cmd = [
        ffmpeg_exe,
        '-i', filepath,
        '-f', 's16le',  # signed 16-bit little-endian PCM
        '-acodec', 'pcm_s16le',
        '-ar', str(target_sr),  # target sample rate
        '-ac', '1',  # mono
        '-loglevel', 'error',
        '-'  # output to stdout
    ]
    
    result = subprocess.run(cmd, capture_output=True)
    
    if result.returncode != 0:
        raise Exception(f"FFmpeg error: {result.stderr.decode()}")
    
    # Convert raw bytes to numpy array
    audio_data = np.frombuffer(result.stdout, dtype=np.int16)
    
    # Normalize to [-1, 1]
    audio = audio_data.astype(np.float32) / 32768.0
    
    return audio, target_sr


def load_audio(filepath: str, target_sr: int = TARGET_SR) -> Tuple[np.ndarray, int]:
    """
    Load audio file and resample to target sample rate.
    Supports WAV, FLAC, MP3, OGG, M4A formats.
    Uses FFmpeg via imageio-ffmpeg as fallback for problematic files.
    """
    try:
        # First try with librosa
        audio, sr = librosa.load(filepath, sr=target_sr, mono=True)
        return audio, sr
    except Exception as e:
        # Fallback: use FFmpeg via imageio-ffmpeg
        try:
            print(f"     ⚠️ librosa failed, trying FFmpeg...")
            audio, sr = load_audio_with_ffmpeg(filepath, target_sr)
            print(f"     ✅ FFmpeg loaded successfully")
            return audio, sr
        except Exception as ffmpeg_error:
            print(f"     ❌ FFmpeg also failed: {ffmpeg_error}")
            raise e


def ensure_same_length(audio1: np.ndarray, audio2: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    """
    Ensure two audio arrays have the same length by trimming.
    """
    min_len = min(len(audio1), len(audio2))
    return audio1[:min_len], audio2[:min_len]


print("✅ Utility functions loaded (with FFmpeg fallback)!")

✅ Utility functions loaded (with FFmpeg fallback)!


## 📊 Step 5: SNR Calculation Functions

In [6]:
def calculate_snr_with_noise(clean_audio: np.ndarray, noise_audio: np.ndarray) -> float:
    """
    Calculate SNR when noise file is available.
    SNR = 10 * log10(signal_power / noise_power)
    """
    clean_audio, noise_audio = ensure_same_length(clean_audio, noise_audio)
    
    signal_power = np.mean(clean_audio ** 2)
    noise_power = np.mean(noise_audio ** 2)
    
    if noise_power < 1e-10:
        return float('inf')
    
    snr = 10 * np.log10(signal_power / noise_power)
    return round(snr, 2)


def estimate_snr_vad(audio: np.ndarray, sr: int) -> float:
    """
    Estimate SNR using Voice Activity Detection (VAD) based on energy.
    This method segments the audio into speech and non-speech regions,
    then estimates SNR from the energy ratio.
    """
    # Frame parameters
    frame_length = int(0.025 * sr)  # 25ms frames
    hop_length = int(0.010 * sr)    # 10ms hop
    
    # Calculate frame energies
    frames = librosa.util.frame(audio, frame_length=frame_length, hop_length=hop_length)
    frame_energies = np.sum(frames ** 2, axis=0)
    
    if len(frame_energies) == 0:
        return 0.0
    
    # Use energy threshold to separate speech/noise
    energy_threshold = np.percentile(frame_energies, 30)  # Bottom 30% as noise
    
    speech_frames = frame_energies[frame_energies > energy_threshold]
    noise_frames = frame_energies[frame_energies <= energy_threshold]
    
    if len(noise_frames) == 0 or len(speech_frames) == 0:
        # Fallback: use overall energy ratio
        mean_energy = np.mean(frame_energies)
        min_energy = np.min(frame_energies) + 1e-10
        snr = 10 * np.log10(mean_energy / min_energy)
        return round(snr, 2)
    
    speech_power = np.mean(speech_frames)
    noise_power = np.mean(noise_frames) + 1e-10
    
    snr = 10 * np.log10(speech_power / noise_power)
    return round(snr, 2)


def calculate_snr(degraded_audio: np.ndarray, sr: int, 
                  noise_path: Optional[str] = None,
                  clean_audio: Optional[np.ndarray] = None) -> float:
    """
    Main SNR calculation function.
    Uses noise file if available, otherwise estimates using VAD.
    """
    if noise_path and os.path.exists(noise_path):
        noise_audio, _ = load_audio(noise_path, sr)
        if clean_audio is not None:
            return calculate_snr_with_noise(clean_audio, noise_audio)
        else:
            return calculate_snr_with_noise(degraded_audio, noise_audio)
    else:
        return estimate_snr_vad(degraded_audio, sr)


print("✅ SNR functions loaded!")

✅ SNR functions loaded!


## 🎯 Step 6: PESQ, STOI, and MOS Calculation

In [7]:
def calculate_pesq_score(clean_audio: np.ndarray, degraded_audio: np.ndarray, 
                         sr: int) -> Optional[float]:
    """
    Calculate PESQ (Perceptual Evaluation of Speech Quality) score.
    PESQ score ranges from -0.5 to 4.5 (higher is better).
    """
    if not PESQ_AVAILABLE:
        return None
    
    try:
        # Ensure same length
        clean_audio, degraded_audio = ensure_same_length(clean_audio, degraded_audio)
        
        # PESQ requires 8000 or 16000 Hz
        if sr not in [8000, 16000]:
            clean_audio = librosa.resample(clean_audio, orig_sr=sr, target_sr=16000)
            degraded_audio = librosa.resample(degraded_audio, orig_sr=sr, target_sr=16000)
            sr = 16000
        
        # Calculate PESQ (wb = wideband for 16kHz, nb = narrowband for 8kHz)
        mode = 'wb' if sr == 16000 else 'nb'
        score = pesq(sr, clean_audio, degraded_audio, mode)
        return round(score, 3)
    
    except Exception as e:
        print(f"  ⚠️ PESQ error: {e}")
        return None


def calculate_stoi_score(clean_audio: np.ndarray, degraded_audio: np.ndarray, 
                         sr: int) -> Optional[float]:
    """
    Calculate STOI (Short-Time Objective Intelligibility) score.
    STOI score ranges from 0 to 1 (higher is better).
    """
    if not STOI_AVAILABLE:
        return None
    
    try:
        # Ensure same length
        clean_audio, degraded_audio = ensure_same_length(clean_audio, degraded_audio)
        
        # Calculate STOI
        score = stoi(clean_audio, degraded_audio, sr, extended=False)
        return round(score, 4)
    
    except Exception as e:
        print(f"  ⚠️ STOI error: {e}")
        return None


def estimate_mos_from_pesq(pesq_score: Optional[float]) -> Optional[float]:
    """
    Estimate MOS from PESQ score using ITU-T P.862.1 mapping.
    MOS ranges from 1 to 5 (higher is better).
    """
    if pesq_score is None:
        return None
    
    # ITU-T P.862.1 mapping formula
    mos = 0.999 + (4.999 - 0.999) / (1 + np.exp(-1.4945 * pesq_score + 4.6607))
    return round(mos, 3)


def calculate_mos(degraded_audio: np.ndarray, sr: int, 
                  pesq_score: Optional[float] = None) -> Optional[float]:
    """
    Calculate MOS (Mean Opinion Score) estimate.
    Uses DNSMOS if available, otherwise falls back to PESQ-to-MOS mapping.
    """
    if DNSMOS_AVAILABLE:
        try:
            score = dnsmos.run(degraded_audio, sr)
            if isinstance(score, dict):
                return round(score.get('mos', score.get('overall', 3.0)), 3)
            return round(float(score), 3)
        except Exception as e:
            print(f"  ⚠️ DNSMOS error: {e}")
    
    # Fallback to PESQ-to-MOS mapping
    return estimate_mos_from_pesq(pesq_score)


print("✅ PESQ/STOI/MOS functions loaded!")

✅ PESQ/STOI/MOS functions loaded!


## ⚡ Step 7: Spike Detection

In [8]:
def detect_spikes(audio: np.ndarray, sr: int, 
                  threshold_ratio: float = SPIKE_THRESHOLD_RATIO,
                  min_duration_ms: float = MIN_SPIKE_DURATION_MS) -> Dict[str, Any]:
    """
    Detect audio spikes (sudden amplitude peaks).
    
    A spike is defined as a sudden amplitude increase that exceeds
    the threshold_ratio times the RMS of the signal.
    
    Returns:
        Dictionary with spike detection results
    """
    # Calculate RMS
    rms = np.sqrt(np.mean(audio ** 2))
    
    # Define spike threshold
    spike_threshold = rms * threshold_ratio
    
    # Find samples exceeding threshold
    spike_mask = np.abs(audio) > spike_threshold
    
    # Count distinct spikes (groups of consecutive samples)
    min_samples = int(min_duration_ms * sr / 1000)
    
    spike_count = 0
    max_spike_amplitude = 0.0
    i = 0
    
    while i < len(spike_mask):
        if spike_mask[i]:
            # Found start of potential spike
            spike_start = i
            while i < len(spike_mask) and spike_mask[i]:
                i += 1
            spike_end = i
            
            # Check if spike duration meets minimum
            if (spike_end - spike_start) >= min_samples or min_samples == 0:
                spike_count += 1
                spike_amplitude = np.max(np.abs(audio[spike_start:spike_end]))
                max_spike_amplitude = max(max_spike_amplitude, spike_amplitude)
        else:
            i += 1
    
    # Determine if audio is spiked (has significant spikes)
    is_spiked = spike_count > 0 and max_spike_amplitude > (rms * threshold_ratio * 1.5)
    
    return {
        "is_spiked": is_spiked,
        "spike_count": spike_count,
        "max_spike_amplitude": round(float(max_spike_amplitude), 6)
    }


print("✅ Spike detection function loaded!")

✅ Spike detection function loaded!


## 📄 Step 8: Report Generation Functions

In [9]:
def convert_to_native_types(obj):
    """
    Recursively convert numpy types to native Python types for JSON serialization.
    """
    if isinstance(obj, dict):
        return {key: convert_to_native_types(value) for key, value in obj.items()}
    elif isinstance(obj, list):
        return [convert_to_native_types(item) for item in obj]
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, (np.float32, np.float64, np.floating)):
        return float(obj)
    elif isinstance(obj, (np.int32, np.int64, np.integer)):
        return int(obj)
    elif isinstance(obj, np.bool_):
        return bool(obj)
    else:
        return obj


def save_json(data: Dict[str, Any], output_path: str) -> None:
    """Save data as JSON file."""
    data = convert_to_native_types(data)
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=2, ensure_ascii=False)


def get_original_audio_id(recorded_name: str) -> str:
    """
    Extract original audio ID from recorded filename.
    Example: 'Audio751_1' -> 'row_751'
    """
    # Extract number from recorded name (e.g., Audio751_1 -> 751)
    import re
    match = re.search(r'Audio(\d+)', recorded_name, re.IGNORECASE)
    if match:
        num = match.group(1)
        return f"row_{num}"
    return None


print("✅ Utility functions loaded!")

✅ Utility functions loaded!


## 🚀 Step 9: Main Processing Function

In [14]:
def process_recorded_audio_with_comparison():
    """
    Process recorded audio files, compare with original audio,
    and update recorded_audio_metadata JSON files with:
    - Signal_to_Noise_Ratio_dB
    - Spiked_Voice_Detection
    - Perceptual_Metrics (PESQ, STOI, MOS) comparing recorded vs original
    """
    print("\n" + "="*70)
    print("🎵 RECORDED AUDIO ANALYSIS WITH ORIGINAL COMPARISON")
    print("="*70)
    
    # Verify folders
    if not os.path.exists(RECORDED_AUDIO_FOLDER):
        print(f"❌ Recorded audio folder not found: {RECORDED_AUDIO_FOLDER}")
        return None
    
    if not os.path.exists(RECORDED_METADATA_FOLDER):
        print(f"❌ Recorded metadata folder not found: {RECORDED_METADATA_FOLDER}")
        return None
    
    if not os.path.exists(ORIGINAL_AUDIO_FOLDER):
        print(f"❌ Original audio folder not found: {ORIGINAL_AUDIO_FOLDER}")
        return None
    
    print(f"\n📂 Recorded Audio: {RECORDED_AUDIO_FOLDER}")
    print(f"📂 Recorded Metadata: {RECORDED_METADATA_FOLDER}")
    print(f"📂 Original Audio (reference): {ORIGINAL_AUDIO_FOLDER}")
    
    # Get list of recorded audio files
    recorded_files = sorted([f for f in os.listdir(RECORDED_AUDIO_FOLDER) 
                             if f.lower().endswith(SUPPORTED_FORMATS)])
    
    print(f"\n📋 Found {len(recorded_files)} recorded audio files to process\n")
    
    successful = 0
    failed = 0
    all_reports = []
    
    for i, rec_file in enumerate(recorded_files, 1):
        rec_name = os.path.splitext(rec_file)[0]
        print(f"\n[{i}/{len(recorded_files)}] Processing: {rec_file}")
        print("=" * 60)
        
        try:
            # Paths
            rec_audio_path = os.path.join(RECORDED_AUDIO_FOLDER, rec_file)
            
            # Find metadata file (handle case sensitivity)
            metadata_path = None
            for meta_file in os.listdir(RECORDED_METADATA_FOLDER):
                if meta_file.lower() == f"{rec_name.lower()}.json":
                    metadata_path = os.path.join(RECORDED_METADATA_FOLDER, meta_file)
                    break
            
            if not metadata_path:
                print(f"  ⚠️ Metadata file not found for: {rec_name}")
                failed += 1
                continue
            
            # Get original audio ID (Audio751_1 -> row_751)
            original_id = get_original_audio_id(rec_name)
            if not original_id:
                print(f"  ⚠️ Could not extract original ID from: {rec_name}")
                failed += 1
                continue
            
            print(f"  📎 Original reference: {original_id}")
            
            # Find original audio file
            original_audio_path = None
            for ext in SUPPORTED_FORMATS:
                potential_path = os.path.join(ORIGINAL_AUDIO_FOLDER, f"{original_id}{ext}")
                if os.path.exists(potential_path):
                    original_audio_path = potential_path
                    break
            
            if not original_audio_path:
                print(f"  ⚠️ Original audio not found: {original_id}")
                failed += 1
                continue
            
            # Load existing metadata
            print(f"  📂 Loading metadata...")
            with open(metadata_path, 'r', encoding='utf-8') as f:
                metadata = json.load(f)
            
            # Load recorded audio
            print(f"  🎵 Loading recorded audio...")
            rec_audio, sr = load_audio(rec_audio_path)
            print(f"     Duration: {len(rec_audio)/sr:.2f}s")
            
            # Load original audio (clean reference)
            print(f"  🎵 Loading original audio (reference)...")
            orig_audio, _ = load_audio(original_audio_path, sr)
            print(f"     Duration: {len(orig_audio)/sr:.2f}s")
            
            # Ensure same length for comparison
            min_len = min(len(rec_audio), len(orig_audio))
            rec_audio_trimmed = rec_audio[:min_len]
            orig_audio_trimmed = orig_audio[:min_len]
            
            # =================================================================
            # Calculate Signal_to_Noise_Ratio_dB (for recorded audio)
            # =================================================================
            print(f"  📈 Calculating SNR...")
            snr = estimate_snr_vad(rec_audio, sr)
            snr = float(snr) if snr is not None else None
            print(f"     SNR: {snr} dB")
            
            # =================================================================
            # Detect Spikes (for recorded audio)
            # =================================================================
            print(f"  ⚡ Detecting spikes...")
            spike_info = detect_spikes(rec_audio, sr)
            spike_info = {
                "is_spiked": bool(spike_info["is_spiked"]),
                "spike_count": int(spike_info["spike_count"]),
                "max_spike_amplitude": float(spike_info["max_spike_amplitude"])
            }
            print(f"     Spiked: {spike_info['is_spiked']}, Count: {spike_info['spike_count']}")
            
            # =================================================================
            # Calculate Perceptual Metrics (comparing recorded vs original)
            # =================================================================
            print(f"  🎯 Calculating PESQ (recorded vs original)...")
            pesq_score = calculate_pesq_score(orig_audio_trimmed, rec_audio_trimmed, sr)
            if pesq_score is not None:
                pesq_score = float(pesq_score)
                print(f"     PESQ: {pesq_score}")
            else:
                print(f"     PESQ: Could not calculate")
            
            print(f"  🔊 Calculating STOI (recorded vs original)...")
            stoi_score = calculate_stoi_score(orig_audio_trimmed, rec_audio_trimmed, sr)
            if stoi_score is not None:
                stoi_score = float(stoi_score)
                print(f"     STOI: {stoi_score}")
            else:
                print(f"     STOI: Could not calculate")
            
            print(f"  📊 Estimating MOS...")
            mos_score = calculate_mos(rec_audio, sr, pesq_score)
            if mos_score is not None:
                mos_score = float(mos_score)
                print(f"     MOS: {mos_score}")
            else:
                print(f"     MOS: Could not calculate")
            
            # =================================================================
            # Update metadata with new fields
            # =================================================================
            metadata["Signal_to_Noise_Ratio_dB"] = snr
            metadata["Spiked_Voice_Detection"] = spike_info
            metadata["Perceptual_Metrics"] = {
                "PESQ_score": pesq_score,
                "STOI_score": stoi_score,
                "MOS_estimate": mos_score,
                "reference_audio": original_id
            }
            
            # Save updated metadata
            save_json(metadata, metadata_path)
            print(f"  💾 Updated: {metadata_path}")
            
            all_reports.append({
                "recorded_name": rec_name,
                "original_id": original_id,
                "snr": snr,
                "spike_info": spike_info,
                "pesq": pesq_score,
                "stoi": stoi_score,
                "mos": mos_score
            })
            successful += 1
            
        except Exception as e:
            print(f"  ❌ Error: {e}")
            import traceback
            traceback.print_exc()
            failed += 1
    
    # Summary
    print("\n" + "="*70)
    print("📊 PROCESSING COMPLETE")
    print("="*70)
    print(f"✅ Successfully processed: {successful} files")
    print(f"❌ Failed: {failed} files")
    print(f"📁 Updated metadata in: {RECORDED_METADATA_FOLDER}")
    print("="*70)
    
    return all_reports


# Run the analysis!
print("🚀 Starting recorded audio analysis with original comparison...")
reports = process_recorded_audio_with_comparison()

🚀 Starting recorded audio analysis with original comparison...

🎵 RECORDED AUDIO ANALYSIS WITH ORIGINAL COMPARISON

📂 Recorded Audio: c:\Users\rayba\Downloads\samsung\recorded_data\recorded_audio
📂 Recorded Metadata: c:\Users\rayba\Downloads\samsung\recorded_data\recorded_audio_metadata
📂 Original Audio (reference): c:\Users\rayba\Downloads\samsung\original_data\audio_file

📋 Found 20 recorded audio files to process


[1/20] Processing: Audio751_1.mp3
  📎 Original reference: row_751
  📂 Loading metadata...
  🎵 Loading recorded audio...
     ⚠️ librosa failed, trying FFmpeg...
     ✅ FFmpeg loaded successfully
     Duration: 2.52s
  🎵 Loading original audio (reference)...
     Duration: 1.62s
  📈 Calculating SNR...
     SNR: 15.600000381469727 dB
  ⚡ Detecting spikes...
     Spiked: False, Count: 0
  🎯 Calculating PESQ (recorded vs original)...
     PESQ: 1.673
  🔊 Calculating STOI (recorded vs original)...
     STOI: 0.239
  📊 Estimating MOS...
     MOS: 1.412
  💾 Updated: c:\Users\ray

## ▶️ Step 10: Run the Analysis

Execute this cell to process all audio files and generate JSON reports.

In [20]:
# Run the analysis - Update existing metadata files!
reports = update_metadata_with_audio_metrics(
    audio_folder=AUDIO_FOLDER,
    metadata_folder=METADATA_FOLDER,
    output_folder=OUTPUT_FOLDER
)


🎵 AUDIO QUALITY ANALYSIS - UPDATE METADATA

📁 Audio folder: c:\Users\rayba\Downloads\samsung\original_data\audio_file
📁 Metadata folder: c:\Users\rayba\Downloads\samsung\original_data\audio_metadata
📁 Output folder: c:\Users\rayba\Downloads\samsung\json_output

📋 Found 20 audio files to process


[1/20] Processing: row_751.flac
  📂 Loading metadata...
  🎵 Loading audio...
     Duration: 1.62s, Sample rate: 16000Hz
  📈 Calculating SNR...
     SNR: 48.540000915527344 dB
  ⚡ Detecting spikes...
     Spiked: False, Count: 0
  💾 Saved: c:\Users\rayba\Downloads\samsung\json_output\row_751_metadata.json
  💾 Updated original: c:\Users\rayba\Downloads\samsung\original_data\audio_metadata\row_751_metadata.json

[2/20] Processing: row_752.flac
  📂 Loading metadata...
  🎵 Loading audio...
     Duration: 5.64s, Sample rate: 16000Hz
  📈 Calculating SNR...
     SNR: 34.709999084472656 dB
  ⚡ Detecting spikes...
     Spiked: False, Count: 0
  💾 Saved: c:\Users\rayba\Downloads\samsung\json_output\row_

## 📋 View Sample Output

Display the first generated report as an example.

In [16]:
# Display sample report
if reports and len(reports) > 0:
    print("\n📄 Sample JSON Output:")
    print("="*50)
    print(json.dumps(reports[0], indent=4))
else:
    print("No reports generated yet. Run the analysis first.")


📄 Sample JSON Output:
{
    "Audio_id/name": "row_0",
    "Signal_to_Noise_Ratio_dB": 15.960000038146973,
    "Perceptual_Metrics": {
        "PESQ_score": null,
        "STOI_score": null,
        "MOS_estimate": null
    },
    "Spiked_Voice_Detection": {
        "is_spiked": false,
        "spike_count": 0,
        "max_spike_amplitude": 0.0
    },
    "Subjective_Benchmarking": {
        "Clarity_Rating": null,
        "Naturalness_Rating": null,
        "Intelligibility_Rating": null
    }
}


## 📊 Summary Statistics

In [17]:
# Calculate summary statistics
if reports and len(reports) > 0:
    snr_values = [r['Signal_to_Noise_Ratio_dB'] for r in reports if r['Signal_to_Noise_Ratio_dB'] is not None]
    spiked_count = sum(1 for r in reports if r['Spiked_Voice_Detection']['is_spiked'])
    
    print("\n📊 Summary Statistics")
    print("="*50)
    print(f"Total files processed: {len(reports)}")
    print(f"\nSNR Statistics:")
    print(f"  - Average: {np.mean(snr_values):.2f} dB")
    print(f"  - Min: {np.min(snr_values):.2f} dB")
    print(f"  - Max: {np.max(snr_values):.2f} dB")
    print(f"\nSpike Detection:")
    print(f"  - Files with spikes: {spiked_count}/{len(reports)}")
else:
    print("No reports available. Run the analysis first.")


📊 Summary Statistics
Total files processed: 20

SNR Statistics:
  - Average: 23.53 dB
  - Min: 8.49 dB
  - Max: 56.81 dB

Spike Detection:
  - Files with spikes: 2/20
